In [159]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [160]:
import gym 
# import gymnasium as gym # Neil added
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [161]:
Discrete(3)

Discrete(3)

In [162]:
Box(0,1,shape=(3,3)).sample()

array([[0.28711832, 0.74327105, 0.62339115],
       [0.71574575, 0.54050624, 0.6754768 ],
       [0.8737637 , 0.18784441, 0.548939  ]], dtype=float32)

In [163]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 57,  26, 116],
       [115, 245, 161],
       [ 75, 150, 169]])

In [164]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([40.836376], dtype=float32))

In [165]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([32.937763], dtype=float32))])

In [166]:
MultiBinary(4).sample()

array([1, 1, 0, 1], dtype=int8)

In [167]:
MultiDiscrete([5,2,2]).sample()

array([2, 1, 1])

# 3. Building an Environment

In [168]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        # super().__init__(self, list([self.observation_space,self.action_space])) # Neil added
        # super().__init__(self) # Neil added
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [169]:
env=ShowerEnv()

In [170]:
env.observation_space.sample()

array([61.98297], dtype=float32)

In [171]:
env.reset()

array([40.])

In [172]:
from stable_baselines3.common.env_checker import check_env

In [173]:
# check_env(env, warn=True)

# 4. Test Environment

In [174]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-36
Episode:2 Score:-54
Episode:3 Score:-60
Episode:4 Score:-50
Episode:5 Score:-22


In [175]:
env.close()

# 5. Train Model

In [176]:
log_path = os.path.join('Training', 'Logs')

In [177]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/nsambhu/anaconda3/envs/RLC2/lib/python3.7/site-packages/stable_baselines3/common/vec_env/patch_gym.py:50: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  "You provided an OpenAI Gym environment. "


In [178]:
model.learn(total_timesteps=400000)

Logging to Training/Logs/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -26.4    |
| time/              |          |
|    fps             | 723      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -27.1       |
| time/                   |             |
|    fps                  | 467         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007838666 |
|    clip_fraction        | 0.0398      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 4.75e-05    |

# 6. Save Model

In [179]:
model.save('PPO')

In [180]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/nsambhu/anaconda3/envs/RLC2/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:71: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,
/home/nsambhu/anaconda3/envs/RLC2/lib/python3.7/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(59.2, 0.9797958971132712)